# Exercices supplémentaires

## Complexité de l'algorithme
Dans cet exercice, il est question d'inverser une matrice de valeurs aléatoires de taille $n*n$ :
* Soumettre le script [`scripts/inv-mat.sh`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/inv-mat.sh) avec la commande `sbatch` :

```Bash
sbatch scripts/inv-mat.sh
```

* Explorer les scripts Python
[`scripts/inv-mat.py`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/inv-mat.py) et 
[`scripts/inv-mat.sh`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/inv-mat.sh) :

```Bash
cat scripts/inv-mat.py
cat scripts/inv-mat.sh
```

* Suivre l'évolution du calcul avec `squeue -u $USER`, environ aux 30 secondes
* Les mesures seront sauvegardées dans le fichier `temps_inv.csv`
* Analyse avec Python, Pandas et Numpy dans le script [`scripts/inv-mat-pred.py`](https://github.com/calculquebec/cip201-serveurs-calcul/blob/main/scripts/inv-mat-pred.py) :

```Bash
module load gcc python scipy-stack

cat scripts/inv-mat-pred.py
python scripts/inv-mat-pred.py temps_inv.csv
```
* Quelle a été la consommation maximale en mémoire-vive?

## Installation de *Benchmark 5D*
Il s'agit d'un code effectuant des multiplications matricielles.
Pour une quantité de données d'ordre $O(n^3)$, le code effectue
un nombre de multiplications et d'additions d'ordre $O(n^5)$.

Pour l'installer dans le dossier `$HOME`:
```Bash
cat scripts/installer/b5D.sh
bash scripts/installer/b5D.sh
```

Afin de tirer profit des différents niveaux de cache-mémoire dans un
ordinateur, le code de *Benchmark 5D* permet de découper les cubes de
données en *blocs* de taille `b1*b2*b3`, et de découper les *blocs* de
données en *prismes* de taille `p1*p2*p3`.
Il est donc requis que `b1>p1`, `b2>p2` et `b3>p3`.
Toutes ces dimensions peuvent être fournies en argument à l'exécutable
`b5D.omp` (compilé avec OpenMP).

La question : quelle combinaison de `{b,p}*{1,2,3}` permet
d'obtenir la meilleure performance lorsque `n=96`?
Pour simplifier le problème : `b1=b2=b3` et `p1=p2=p3`.

### Utiliser GLOST
On se rappelle que GLOST fonctionne avec une liste de commandes
indépendantes. Pour générer et valider cette liste :
```Bash
cat scripts/extra/b5D-gen-cas.sh
bash scripts/extra/b5D-gen-cas.sh

cat $SCRATCH/cas-b5D.txt
```

Éditez le fichier `scripts/extra/b5D-glost.sh` dans le but
d'utiliser GLOST :
1. Demandez 4 tâches Slurm par noeud
1. Faites charger le module `glost/0.3.1` après le module GCC
1. Construisez la commande de démarrage de GLOST avec `srun`
   et `$SCRATCH/cas-b5D.txt` pour le fichier de commandes.

Une fois que ce script de tâche est complété, vous pouvez le soumettre
à l'ordonnanceur :
```Bash
sbatch scripts/extra/b5D-glost.sh
```

Dans le fichier de sortie Slurm, vous devriez pouvoir identifier la
combinaison optimale pour `b` et `p` en considérant les vitesses.

### Utiliser GNU Parallel
Éditez le fichier `scripts/extra/b5D-parallel.sh` dans le but
d'utiliser GPU Parallel :
1. Demandez 4 processeurs pour une et une seule tâche Slurm
1. Demandez 2000M de mémoire-vive par processeur
1. Faites en sorte que `./b5D.omp` soit appelé par GNU Parallel
1. Tous les paramètres `-b*` auront la même valeur, mais les
   différentes valeurs à tester sont : `48 32 24`
1. Tous les paramètres `-p*` auront la même valeur, mais les
   différentes valeurs à tester sont : `16 12 8 6`

Une fois que ce script de tâche est complété, vous pouvez le soumettre
à l'ordonnanceur :
```Bash
sbatch scripts/extra/b5D-parallel.sh
```

Dans le fichier de sortie Slurm, vous devriez pouvoir identifier la
combinaison optimale pour `b` et `p` en considérant les vitesses.

### Analyse des ressources utilisées
* Utilisez la commande `sacct` pour obtenir le détail de vos dernières
  tâches.
* Utilisez la commande `seff` avec un numéro de tâche pour obtenir ses
  ressources consommées : temps CPU, quantité de mémoire-vive et
  pourcentages d'efficacité.

### Problème plus grand
Sachant quelles sont les meilleurs paramètres pour `b` et `p`,
construisez un nouveau script de tâche pour tester trois (3)
lancements successifs de la commande `./b5D.omp` avec `-n 144` :

* Réservez une tâche Slurm de un processeur
* Réservez suffisamment de mémoire-vive, mais pas trop
* Réservez suffisamment de temps de calcul
* Utilisez la commande `time -p` pour mesurer le temps de chacune des
  trois exécutions.

1. Une fois le script complété, soumettez-le et vérifiez les résultats.
   En cas d'erreur, faites les corrections et essayez à nouveau
1. Vérifiez les ressources utilisées (temps CPU, mémoire, etc.)